In [3]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.stats import t as t_distribution
np.random.seed(42)

In [ ]:
Normal_cdf = norm.cdf

def BS_CALL(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * Normal_cdf(d1) - K * np.exp(-r*T)* Normal_cdf(d2)

def BS_PUT(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma* np.sqrt(T)
    return K*np.exp(-r*T)*Normal_cdf(-d2) - S*Normal_cdf(-d1)

In [ ]:
#MC для ванили
def monte_carlo_vanilla_option(S, K, T, r, sigma, N=1000000, option_type='call', q=0.0):

    Z = np.random.normal(size=N)
    ST = S * np.exp((r - q - 0.5*sigma**2)*T + sigma * np.sqrt(T) * Z)
    if option_type == 'call':
        payoffs = np.maximum(ST - K, 0)
    else:
        payoffs = np.maximum(K - ST, 0)

    price = np.exp(-r * T) * np.mean(payoffs)
    return price


In [ ]:
S = 1.0
K = 1.00714286
T = 1.01020408
r = 0.045
sigma_imp = 0.475627
sigam_vol = 0.453407
q=0.0
N=1000000

# Монте-Карло call и put
mc_call = monte_carlo_vanilla_option(S, K, T, r, sigam_vol, N, option_type='call', q=q)
mc_put = monte_carlo_vanilla_option(S, K, T, r, sigam_vol, N, option_type='put', q=q)

# Black_Scholes call and put
bs_call = BS_CALL(S, K, T, r, sigma_imp)
bs_put = BS_PUT(S, K, T, r, sigma_imp)


print(f"MC Call: {mc_call:.4f}")

print(f"MC Put: {mc_put:.4f}")

print(f"BS Call: {bs_call:.4f}")
print(f"BS Put: {bs_put:.4f}")




при фикс Т строить сплайн по сигма и хранить коэфы
при новом сигма берем T, для него слпайн берем



In [9]:
dfs = pd.read_excel('/content/sigma_dupir.xlsx')

In [10]:
dfs

,strike,T,sigma_dup
0,0.650000,0.5,0.389648
1,0.650701,0.5,0.389587
2,0.651401,0.5,0.389494
3,0.652102,0.5,0.389402
4,0.652803,0.5,0.389309
...,...,...,...
999995,1.347197,1.5,0.505828
999996,1.347898,1.5,0.505832
999997,1.348599,1.5,0.505837
999998,1.349299,1.5,0.505842


In [11]:
from scipy.interpolate import InterpolatedUnivariateSpline


# cписок уникальных T
T_vals = np.sort(dfs['T'].dropna().unique())

# cловарь: T -> spline(strike)
splines_by_T = {}

for T in T_vals:
    sub = dfs[dfs['T'] == T]
    strikes = sub['strike'].values
    sigmas = sub['sigma_dup'].values

    # сорт. по strike
    idx = np.argsort(strikes)
    strikes_sorted = strikes[idx]
    sigmas_sorted = sigmas[idx]

    spline = InterpolatedUnivariateSpline(strikes_sorted, sigmas_sorted, k=3, ext='const')
    splines_by_T[T] = spline

def get_sigma(K, T, splines_by_T, T_vals):
    T_nearest = T_vals[np.abs(T_vals - T).argmin()]
    spline = splines_by_T[T_nearest]
    return float(spline(K))


In [29]:
def monte_carlo_asian_call_fullgrid(S0, K, splines_by_T, T_vals, T_full, tenors, r, N=10000, q=0.99):


    M = len(T_full)
    S = np.full(N, S0)
    S_sampled = np.zeros((N, len(tenors)))
    idxs_sample = set(np.searchsorted(T_full, tenors))
    curr_sample_idx = 0

    for j in range(1, M):
        dt = T_full[j] - T_full[j-1]
        sigmas = np.array([get_sigma(s, T_full[j], splines_by_T, T_vals) for s in S])
        Z = np.random.normal(size=N)
        S = S * np.exp((r - 0.5 * sigmas**2) * dt + sigmas * np.sqrt(dt) * Z)
        if j in idxs_sample:
            S_sampled[:, curr_sample_idx] = S
            curr_sample_idx += 1


    A = S_sampled.mean(axis=1)
    payoff = np.maximum(A - K, 0)
    price = np.exp(-r * tenors[-1]) * payoff.mean()
    error = np.exp(-r * tenors[-1]) * payoff.std() * t_distribution.ppf(q, N-1) / np.sqrt(N)
    return price, error


T_full = np.sort(dfs['T'].unique())
M_full = len(T_full)
m = 12

idxs = np.linspace(0, M_full-1, m, dtype=int)
tenors = T_full[idxs]


S0 = 1.0
K = 1.1
r = 0.045
N = 10000
q = 0.99

price, error = monte_carlo_asian_call_fullgrid(
    S0, K, splines_by_T, T_vals, T_full, tenors, r, N=N, q=q
)
print(f"Asian MC call price: {price:.5f} ± {error:.5f}")


Asian MC call price: 0.04328 ± 0.00280


In [33]:
def Forward_starting_call_localvol(
        S0, T1, T2, splines_by_T, T_vals, time_grid, r, N=10000, q=0.99):


    idx_T1 = np.searchsorted(time_grid, T1)
    idx_T2 = np.searchsorted(time_grid, T2)

    S = np.full(N, S0)
    t_prev = time_grid[0]

    for t in time_grid[1:idx_T1+1]:
        dt = t - t_prev
        sigmas = np.array([get_sigma(s, t, splines_by_T, T_vals) for s in S])
        Z = np.random.normal(size=N)
        S = S * np.exp((r - 0.5 * sigmas**2) * dt + sigmas * np.sqrt(dt) * Z)
        t_prev = t
    S_T1 = S.copy()

    for t in time_grid[idx_T1+1:idx_T2+1]:
        dt = t - t_prev
        sigmas = np.array([get_sigma(s, t, splines_by_T, T_vals) for s in S])
        Z = np.random.normal(size=N)
        S = S * np.exp((r - 0.5 * sigmas**2) * dt + sigmas * np.sqrt(dt) * Z)
        t_prev = t
    S_T2 = S.copy()

    Call = np.maximum(S_T2 - S_T1, 0)
    price = np.exp(-r * T2) * Call.mean()
    error = np.exp(-r * T2) * Call.std() * t_distribution.ppf(q, N-1) / np.sqrt(N)
    return price, error



S0 = 1.0
T1 = 1.0
T2 = 1.5
r = 0.045
N = 10000
q = 0.99

time_grid = np.sort(dfs['T'].unique())

price, error = Forward_starting_call_localvol(
    S0, T1, T2, splines_by_T, T_vals, time_grid, r, N=N, q=q
)
print(f"Forward starting call (MC, local vol): {price:.5f} ± {error:.5f}")


Forward starting call (MC, local vol): 0.14507 ± 0.00636


In [35]:
import numpy as np


def trajectories_localvol(
    r, splines_by_T1, splines_by_T2, T_vals, corr, S_0, time_grid, N
):
    m = len(time_grid)
    S_1 = np.zeros((N, m))
    S_2 = np.zeros((N, m))
    S_1[:, 0] = S_0
    S_2[:, 0] = S_0
    t_prev = time_grid[0]
    for i in range(1, m):
        t = time_grid[i]
        dt = t - t_prev
        # лок вола для каждого пути
        sig1 = np.array([get_sigma(S_1[j, i-1], t, splines_by_T1, T_vals) for j in range(N)])
        sig2 = np.array([get_sigma(S_2[j, i-1], t, splines_by_T2, T_vals) for j in range(N)])
        Z1 = np.random.normal(size=N)
        Z2 = np.random.normal(size=N)
        Z1_corr = Z1
        Z2_corr = corr * Z1 + np.sqrt(1 - corr ** 2) * Z2
        S_1[:, i] = S_1[:, i-1] * np.exp((r - 0.5 * sig1**2) * dt + sig1 * np.sqrt(dt) * Z1_corr)
        S_2[:, i] = S_2[:, i-1] * np.exp((r - 0.5 * sig2**2) * dt + sig2 * np.sqrt(dt) * Z2_corr)
        t_prev = t
    return S_1, S_2


def detect_TimeCrossing(S_1, S_2, TB, B, time_grid):
    N, m = S_1.shape
    time_high = np.zeros(N)
    time_low = np.zeros(N)
    for i in range(N):
        for j in range(m):
            if S_1[i, j] >= TB and S_2[i, j] >= TB:
                time_high[i] = time_grid[j]
                break
            if S_1[i, j] <= B or S_2[i, j] <= B:
                time_low[i] = 1
    return time_high, time_low

def payoff(S_1, S_2, time_high, time_low, K, time_grid, coupon_dates, c, r):
    N, m = S_1.shape
    discounted_price = np.zeros(N)
    for i in range(N):
        discounted_coupon = 0
        discounted_strike = 0
        if time_high[i] > 0:
            coupon_times = coupon_dates[coupon_dates <= time_high[i]]
            for t in coupon_times:
                discounted_coupon += np.exp(-r * t) * c
            discounted_strike = np.exp(-r * time_high[i]) * K
            discounted_price[i] = discounted_coupon + discounted_strike
        elif time_high[i] == 0 and time_low[i] > 0:
            for t in coupon_dates:
                discounted_coupon += np.exp(-r * t) * c
            S_T = min(S_1[i, -1], S_2[i, -1])
            discounted_strike_correction = np.exp(-r * time_grid[-1]) * (K - max(K - S_T, 0))
            discounted_price[i] = discounted_coupon + discounted_strike_correction
        else:
            for t in coupon_dates:
                discounted_coupon += np.exp(-r * t) * c
            discounted_strike = np.exp(-r * time_grid[-1]) * K
            discounted_price[i] = discounted_coupon + discounted_strike
    return np.mean(discounted_price)

def find_c(S_1, S_2, time_high, time_low, K, time_grid, coupon_dates, r, V0, c_min=0, c_max=10, tol=0.01, max_iter=1000):
    for _ in range(max_iter):
        c = (c_min + c_max) / 2
        R = payoff(S_1, S_2, time_high, time_low, K, time_grid, coupon_dates, c, r)
        if abs(R - V0) < tol:
            return c
        elif R < V0:
            c_min = c
        else:
            c_max = c
    return (c_min + c_max) / 2




S_0 = 1.0
TB = 1.05
B = 0.8
K = 1.0
r = 0.045
corr = 0.3
V_0 = 1.0
N = 10000


splines_by_T1 = splines_by_T
splines_by_T2 = splines_by_T
T_vals = np.sort(dfs['T'].unique())
time_grid = np.sort(dfs['T'].unique())
coupon_dates = time_grid[::10]


S_1, S_2 = trajectories_localvol(r, splines_by_T1, splines_by_T2, T_vals, corr, S_0, time_grid, N)
time_high, time_low = detect_TimeCrossing(S_1, S_2, TB, B, time_grid)
c_optimal = find_c(S_1, S_2, time_high, time_low, K, time_grid, coupon_dates, r, V_0)

print(f"Оптимальное значение c: {c_optimal:.5f}")


Оптимальное значение c: 0.00366
